In [100]:
import cv2
import os
import json
import matplotlib.pyplot as plt

dir = '../cropped_monet_signatures' # signature pictures dir

if os.path.isdir(dir):
    content = os.listdir(dir)
    
    for item in content:
        if item.find('.jpg') > 0:
            image = cv2.imread(f'{dir}/{item}', cv2.IMREAD_GRAYSCALE)
            
            # GENERATING :SIFT: METRICS
            sift = cv2.SIFT_create()

            keypoints, descriptors = sift.detectAndCompute(image, None)

            # print(str(keypoints))
            # SAVING .json FILES
            keypoint_list = list(keypoints)

            for i in range(len(keypoint_list)):
                keypoint_list[i] = str(keypoint_list[i]) # [15:31]

            payload = {
                'keypoints': keypoint_list,
                'descriptors': descriptors.tolist()
            }

            with open(f'{dir}/{item.replace(".jpg", "")}.json', 'w') as file:
                json.dump(payload, file)

In [101]:
if os.path.isdir(dir):
    content = os.listdir(dir)
    
    for item in content:
        if item.find('2.json') > 0:
            image = cv2.imread(f'{dir}/{item.split(".")[0]}.jpg', cv2.IMREAD_GRAYSCALE)

            keypoint_list = sift_payload['keypoints']
            
            with open(f'{dir}/{item}', 'r') as arquivo:
                sift_payload = json.load(arquivo)

            print(keypoint_list)

            # image_with_keypoints = cv2.drawKeypoints(image, keypoint_list, None)

            # plt.imshow(image_with_keypoints, cmap='gray')
            # plt.axis('off')  # Para não mostrar os eixos
            # plt.show()

['< cv2.KeyPoint 000002006F481360>', '< cv2.KeyPoint 000002006F481240>', '< cv2.KeyPoint 000002006F652420>', '< cv2.KeyPoint 000002006F652450>', '< cv2.KeyPoint 000002006BCA1BA0>', '< cv2.KeyPoint 000002006BCA1810>', '< cv2.KeyPoint 000002006BCA17E0>', '< cv2.KeyPoint 000002007FA54510>', '< cv2.KeyPoint 000002007FA54390>', '< cv2.KeyPoint 000002007FA546C0>', '< cv2.KeyPoint 00000200700C9480>', '< cv2.KeyPoint 00000200700C9600>', '< cv2.KeyPoint 000002006F731C30>', '< cv2.KeyPoint 000002006F731CC0>', '< cv2.KeyPoint 000002006F731CF0>', '< cv2.KeyPoint 000002006F731D20>', '< cv2.KeyPoint 000002006F731D50>', '< cv2.KeyPoint 000002006F731D80>', '< cv2.KeyPoint 000002006F731DB0>', '< cv2.KeyPoint 000002006F731E70>', '< cv2.KeyPoint 000002006F731F30>', '< cv2.KeyPoint 000002006F731F90>', '< cv2.KeyPoint 000002006F731F60>', '< cv2.KeyPoint 000002006F731C90>', '< cv2.KeyPoint 000002006F7311E0>', '< cv2.KeyPoint 000002006F731DE0>', '< cv2.KeyPoint 000002006F731E40>', '< cv2.KeyPoint 000002006F7